In [ ]:
!pip install PyPDF2
!pip install pdfminer 
!pip install pdfminer.six
!pip install tabula-py
!pip3 install pdf_layout_scanner

In [8]:
from PyPDF2 import PdfReader

In [ ]:
reader = PdfReader('./OneDrive_1_10-4-2024/JobThai - 1032276_ฐาณภูมิ์_ธีรสีหรัตน์ - Application.pdf')

for page in reader.pages:
    text = page.extract_text()
    print(text)

In [ ]:
import tabula
tables = tabula.read_pdf('./OneDrive_1_10-4-2024/JobThai - 1046698_จำเริญ_ศรีบุญเรือง - Application.pdf', pages='all')
for i, table in enumerate(tables, start=1):
    print(f"Table {i}:")
    print(table)
    print("-----------------------")

In [ ]:
from pdf_layout_scanner import layout_scanner
toc=layout_scanner.get_toc('./OneDrive_1_10-4-2024/JobThai - 1032276_ฐาณภูมิ์_ธีรสีหรัตน์ - Application.pdf')
print(toc)

pages=layout_scanner.get_pages('./OneDrive_1_10-4-2024/JobThai - 1032276_ฐาณภูมิ์_ธีรสีหรัตน์ - Application.pdf')
print(pages)

In [ ]:
! pip install azure-ai-formrecognizer

In [26]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import os
from dotenv import load_dotenv

load_dotenv()

AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT=os.environ.get("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
AZURE_DOCUMENT_INTELLIGENCE_KEY=os.environ.get("AZURE_DOCUMENT_INTELLIGENCE_KEY")

In [27]:
PDF_HEADERS = {
    "title": "h1",
    "sectionHeading": "h2"
}

endpoint = AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT
credential = AzureKeyCredential(AZURE_DOCUMENT_INTELLIGENCE_KEY)
form_recognizer_client = DocumentAnalysisClient(endpoint, credential)
def table_to_html(table):
    table_html = "<table>"
    rows = [sorted([cell for cell in table.cells if cell.row_index == i], key=lambda cell: cell.column_index) for i in range(table.row_count)]
    for row_cells in rows:
        table_html += "<tr>"
        for cell in row_cells:
            tag = "th" if (cell.kind == "columnHeader" or cell.kind == "rowHeader") else "td"
            cell_spans = ""
            if cell.column_span > 1: cell_spans += f" colSpan={cell.column_span}"
            if cell.row_span > 1: cell_spans += f" rowSpan={cell.row_span}"
            table_html += f"<{tag}{cell_spans}>{html.escape(cell.content)}</{tag}>"
        table_html +="</tr>"
    table_html += "</table>"
    return table_html

def extract_pdf_content(file_path, form_recognizer_client, use_layout=False): 
    offset = 0
    page_map = []
    model = "prebuilt-layout" if use_layout else "prebuilt-read"
    with open(file_path, "rb") as f:
        poller = form_recognizer_client.begin_analyze_document(model, document = f)
    form_recognizer_results = poller.result()
    # (if using layout) mark all the positions of headers
    roles_start = {}
    roles_end = {}
    for paragraph in form_recognizer_results.paragraphs:
        if paragraph.role!=None:
            para_start = paragraph.spans[0].offset
            para_end = paragraph.spans[0].offset + paragraph.spans[0].length
            roles_start[para_start] = paragraph.role
            roles_end[para_end] = paragraph.role

    for page_num, page in enumerate(form_recognizer_results.pages):
        tables_on_page = [table for table in form_recognizer_results.tables if table.bounding_regions[0].page_number == page_num + 1]

        # (if using layout) mark all positions of the table spans in the page
        page_offset = page.spans[0].offset
        page_length = page.spans[0].length
        table_chars = [-1]*page_length
        for table_id, table in enumerate(tables_on_page):
            for span in table.spans:
                # replace all table spans with "table_id" in table_chars array
                for i in range(span.length):
                    idx = span.offset - page_offset + i
                    if idx >=0 and idx < page_length:
                        table_chars[idx] = table_id

        # build page text by replacing charcters in table spans with table html and replace the characters corresponding to headers with html headers, if using layout
        page_text = ""
        added_tables = set()
        for idx, table_id in enumerate(table_chars):
            if table_id == -1:
                position = page_offset + idx
                if position in roles_start.keys():
                    role = roles_start[position]
                    if role in PDF_HEADERS:
                        page_text += f"<{PDF_HEADERS[role]}>"
                if position in roles_end.keys():
                    role = roles_end[position]
                    if role in PDF_HEADERS:
                        page_text += f"</{PDF_HEADERS[role]}>"

                page_text += form_recognizer_results.content[page_offset + idx]
                
            elif not table_id in added_tables:
                page_text += table_to_html(tables_on_page[table_id])
                added_tables.add(table_id)

        page_text += " "
        page_map.append((page_num, offset, page_text))
        offset += len(page_text)

    full_text = "".join([page_text for _, _, page_text in page_map])
    return full_text

In [72]:
extracted_text = extract_pdf_content('./OneDrive_1_10-4-2024/pid_redacted/356LW841M3P0.pdf', form_recognizer_client, False)

In [73]:
import os
from openai import AzureOpenAI

AZURE_OPENAI_PREVIEW_API_VERSION=os.environ.get("AZURE_OPENAI_PREVIEW_API_VERSION")
AZURE_OPENAI_API_KEY=os.environ.get("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT=os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_MODEL=os.environ.get("AZURE_OPENAI_MODEL")

client = AzureOpenAI(
    api_version= AZURE_OPENAI_PREVIEW_API_VERSION,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

In [74]:
completion = client.chat.completions.create(
    model=AZURE_OPENAI_MODEL,
    messages=[
        {
            "role": "system",
            "content": "You are HR department assistant your job is to write a short summary this candidate data focusing on the data that important to hiring process"
        },
        {
            "role": "user",
            "content": extracted_text,
        },
    ],
)

In [ ]:
print(completion.to_json())

#### censor data

In [ ]:
! pip install pymupdf reportlab

In [56]:
import fitz  # PyMuPDF

# Function to redact a specific area on a PDF page based on coordinates
def redact_area(input_pdf, output_pdf, page_num, coordinates):
    # Open the PDF file
    doc = fitz.open(input_pdf)
    
    # Load the specified page
    page = doc.load_page(page_num)
    
    # Coordinates: (x0, y0, x1, y1) defining the rectangle to redact
    for coord in coordinates:
        x0, y0, x1, y1 = coord
        # Create a rectangle object with these coordinates
        rect = fitz.Rect(x0, y0, x1, y1)
        
        # Add the redaction annotation
        page.add_redact_annot(rect, fill=(0, 0, 0))  # Black out the rectangle
        
    # Apply all redactions on the page
    page.apply_redactions()

    # Save the modified PDF
    doc.save(output_pdf)
    doc.close()

# Example usage:
input_pdf = "./OneDrive_1_10-4-2024/JobThai - 1032276_ฐาณภูมิ์_ธีรสีหรัตน์ - Application.pdf"
output_pdf = "output.pdf"
page_num = 0  # Redact the first page

# Define the coordinates of the areas to redact (x0, y0, x1, y1)
coordinates = [
    (50, 75, 300, 190),  # First area to redact
    (50, 210, 300, 340),  # Another area to redact
]

redact_area(input_pdf, output_pdf, page_num, coordinates)

In [ ]:
import os
import random
import string

input_folder = r'./OneDrive_1_10-4-2024'
output_folder = r'./OneDrive_1_10-4-2024/pid_redacted'
coordinates = [
    (50, 75, 300, 190),  # First area to redact
    (50, 210, 300, 340),  # Another area to redact
]

for file in os.listdir(input_folder):
    input_path = os.path.join(input_folder, file)
    output_path = os.path.join(output_folder, f"{''.join(random.choices(string.ascii_uppercase + string.digits, k=12))}.pdf")
    try :
        redact_area(input_pdf=input_path, output_pdf=output_path, page_num=0, coordinates=coordinates)
    except :
        continue
    print(f"{file} redacted complete")